Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Getting Started

In this tutorial, we demonstrate some of the basic features of TorchGeo and show how easy it is to use if you're already familiar with other PyTorch domain libraries like torchvision.

It's recommended to run this notebook on Google Colab if you don't have your own GPU. Click the "Open in Colab" button above to get started.

## Setup

First, we install TorchGeo.

In [ ]:
%pip install torchgeo

## Imports

Next, we import TorchGeo and any other libraries we need.

In [13]:
import os
import tempfile

from torch.utils.data import DataLoader

from torchgeo.datasets import NAIP, ChesapeakeDE, stack_samples
from torchgeo.datasets.utils import download_url
from torchgeo.samplers import RandomGeoSampler

## Datasets

For this tutorial, we'll be using imagery from the [National Agriculture Imagery Program (NAIP)](https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/) and labels from the [Chesapeake Bay High-Resolution Land Cover Project](https://www.chesapeakeconservancy.org/conservation-innovation-center/high-resolution-data/land-cover-data-project/). First, we manually download a few NAIP tiles and create a PyTorch Dataset.

In [4]:
data_root = tempfile.gettempdir()
naip_root = os.path.join(data_root, "naip")
naip_url = (
    "https://naipeuwest.blob.core.windows.net/naip/v002/de/2018/de_060cm_2018/38075/"
)
tiles = [
    "m_3807511_ne_18_060_20181104.tif",
    "m_3807511_se_18_060_20181104.tif",
    "m_3807512_nw_18_060_20180815.tif",
    "m_3807512_sw_18_060_20180815.tif",
]
for tile in tiles:
    download_url(naip_url + tile, naip_root)

naip = NAIP(naip_root)

  0%|          | 0/581769974 [00:00<?, ?it/s]

  0%|          | 0/590502354 [00:00<?, ?it/s]

  0%|          | 0/574468665 [00:00<?, ?it/s]

  0%|          | 0/569661931 [00:00<?, ?it/s]

Next, we tell TorchGeo to automatically download the corresponding Chesapeake labels.

In [9]:
chesapeake_root = os.path.join(data_root, "chesapeake")

chesapeake = ChesapeakeDE(chesapeake_root, crs=naip.crs, res=naip.res, download=True)

Files already downloaded and verified


Finally, we create an IntersectionDataset so that we can automatically sample from both GeoDatasets simultaneously.

In [10]:
dataset = naip & chesapeake

## Sampler

Unlike typical PyTorch Datasets, TorchGeo GeoDatasets are indexed using lat/long/time bounding boxes. This requires us to use a custom GeoSampler instead of the default sampler/batch_sampler that comes with PyTorch.

In [12]:
sampler = RandomGeoSampler(naip, size=1000, length=10)

## DataLoader

Now that we have a Dataset and Sampler, we can combine these into a single DataLoader.

In [16]:
dataloader = DataLoader(dataset, sampler=sampler, collate_fn=stack_samples)

## Training

Other than that, the rest of the training pipeline is the same as it is for torchvision.

In [18]:
for sample in dataloader:
    image = sample["image"]
    target = sample["mask"]